In [1]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough,RunnableMap


c:\Users\HP\anaconda3\envs\groqbot2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install langchain-groq langchain-community faiss-cpu 

In [3]:
!pip install langchain-groq


In [4]:
from dotenv import load_dotenv
load_dotenv()
import os



In [36]:
groq_api_key=os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile"  # or "llama-3.1-8b-instant" / "llama-3.2-3b-preview"
)

In [7]:
response = llm.invoke("What is LangChain?")
print(response.content)

LangChain is an open-source framework designed to help developers build applications that utilize large language models (LLMs) more efficiently. It was created to streamline the process of integrating LLMs into various projects, providing a set of tools and libraries that simplify tasks such as prompting, chaining, and embedding models.

LangChain aims to address some of the key challenges associated with working with LLMs, such as:

1. **Prompt engineering**: Crafting effective prompts that elicit desired responses from the model.
2. **Model chaining**: Combining multiple models to achieve more complex tasks or improve overall performance.
3. **Embedding models**: Integrating LLMs into larger applications, such as chatbots, virtual assistants, or content generation tools.

By providing a standardized framework for working with LLMs, LangChain enables developers to focus on building innovative applications rather than struggling with the underlying complexities of the models themselves

In [8]:
file_path = "DATA"
db_path="vectorstore/db_faiss"

In [9]:
#create vector database
def create_vector_db():
    loader = DirectoryLoader(file_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    print(f"Number of documents loaded: {len(documents)}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts = text_splitter.split_documents(documents)
    print(f"Number of documents: {len(texts)}")

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(texts, embeddings)
    # store the vector store to disk
    vector_store.save_local(db_path)# save vector store to disk
    
    print("Vector store created successfully.")
    return vector_store

In [10]:
create_vector_db()

Number of documents loaded: 759
Number of documents: 6973


C:\Users\HP\AppData\Local\Temp\ipykernel_23304\1972567339.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Vector store created successfully.


In [11]:
loader = DirectoryLoader(file_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
print(f"Number of text chunks: {len(texts)}")

Number of text chunks: 6973


In [12]:
def load_vector_db():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector DB loaded successfully!")
    return vector_store

In [13]:
#  Initialize Groq Client (No LangChain)
from groq import Groq
def initialize_groq_client():
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("Missing GROQ_API_KEY in .env file")

    client = Groq(api_key=groq_api_key)
    print("Groq client initialized successfully!")
    return client


In [40]:
def initialize_llm():
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("Missing GROQ_API_KEY in .env file")

    llm = ChatGroq(
        api_key=os.getenv("GROQ_API_KEY"),
        model="llama-3.3-70b-versatile",
        temperature=0.2,
    )
    return llm

In [ ]:
def rag_chain()

In [28]:
#Step 3: Build RAG pipeline using Runnables (modern LangChain)

def create_rag_pipeline():
    db = load_vector_db()
    retriever = db.as_retriever(search_kwargs={"k": 4})
    llm = initialize_llm()

    prompt = ChatPromptTemplate.from_template("""
    You are a knowledgeable and friendly medical assistant.
    Use the provided context to answer the question accurately and clearly.
    Your answers should be concise and factual.
    Do not make up any information.
    
    Context:
    {context}

    Question:
    {question}

    Helpful and concise answer:
    """)

    # ⚙️ Build Runnable pipeline: Retriever → Prompt → LLM
    rag_pipeline = (
        RunnableMap({
            "context": retriever,
            "question": RunnablePassthrough()
        })
        | prompt
        | llm
    )

    return rag_pipeline


In [30]:
 #Step 4: Run Query
if __name__ == "__main__":
    # Run this once to create the vector DB
    # create_vector_db()

    rag_pipeline = create_rag_pipeline()

    user_query = "CAN I DIE?"
    response = rag_pipeline.invoke(user_query)

    print("\nChatbot Response:\n")
    print(response.content)

Vector DB loaded successfully!

Chatbot Response:

I'm here to provide you with helpful information. According to the context, death can occur due to various reasons such as severe illness, drug overdose, or failure of major organs. However, the decision to die or let a loved one die can be complex and is often determined by medical professionals and the individual's living will. If you're experiencing any distress or have concerns about your health, I encourage you to reach out to a medical professional for guidance and support.


In [38]:
# Enhance RAG pipeline with conversation memory and correct retriever input

chat_history = []  # global conversation memory

def create_rag_pipeline():
    """Create RAG pipeline with chat memory and proper retriever input."""
    db = load_vector_db()
    retriever = db.as_retriever(search_kwargs={"k": 4})
    llm = initialize_llm()

    prompt = ChatPromptTemplate.from_template("""
You are a knowledgeable and friendly **medical assistant**.
Answer the question **accurately and concisely**, using only the provided context.
If the context does not contain the answer, say: "I'm not sure based on the available information."

Previous conversation (for context):
{chat_memory}

Context:
{context}

Question:
{question}

Answer:
""")

    def add_memory(inputs):
        """Attach last few exchanges from chat history."""
        last_turns = chat_history[-3:]  # keep only last 3 exchanges
        formatted = "\n".join([f"User: {u}\nBot: {b}" for u, b in last_turns])
        inputs["chat_memory"] = formatted
        return inputs

    def retrieve_context(inputs):
        """Retrieve context using retriever with correct string input."""
        question = inputs["question"]
        docs = retriever.invoke(question)
        context = "\n\n".join([doc.page_content for doc in docs])
        inputs["context"] = context
        return inputs

    #  Runnable pipeline flow: Add memory → Retrieve context → Prompt → LLM
    rag_pipeline = (
        RunnablePassthrough()
        | add_memory
        | retrieve_context
        | prompt
        | llm
    ) # Runnable passthrough acts as a placeholder it pass the input to output without modification

    #print("Fixed RAG pipeline with memory and proper retriever flow.")
    return rag_pipeline


def chat_with_bot(question, rag_pipeline):
    """Ask a question, update memory, and return bot’s response."""
    response = rag_pipeline.invoke({"question": question})
    answer = response.content if hasattr(response, "content") else response
    chat_history.append((question, answer))
    return answer

In [39]:
rag_pipeline = create_rag_pipeline()

print(chat_with_bot("What are the symptoms of pneumonia?", rag_pipeline))
print(chat_with_bot("How can it be prevented?", rag_pipeline))


Vector DB loaded successfully!
The symptoms of pneumonia include fever, difficulty breathing, bluish lips or fingernails, a cough that brings up greenish-yellow or grayish sputum, and other symptoms such as skin rash, swollen glands, or whitish spots on the tonsils or throat.
It appears there are multiple diseases being discussed. Based on the context, prevention methods include:

1. Good hygiene and nutrition, avoiding crowds or people with active infections.
2. Treating all infected cases to eliminate the source of infection, washing hands, keeping fingernails short and clean, avoiding scratching the anal area, and taking early morning showers.
3. Preventing onset of underlying disorders, prenatal testing, genetic counseling, refraining from injury-causing activities, and good hygiene, especially skin and mouth care.
4. Preventive antibiotics and avoiding crowds or people with infections. 

Note: The specific disease is not mentioned, so these are general prevention methods mentioned